In [ ]:
!nvidia-smi

In [ ]:
!pip install -qU transformers
!pip install -qU hazm

In [ ]:
import hazm
normalizer = hazm.Normalizer(persian_numbers=False)

def normalize_input(text):
    text = normalizer.normalize(text)
    return text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import os
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm

In [ ]:
class Data_load:
  def __init__(self,max_length, step):
    self.vocab = []
    self.max_length = max_length
    self.step = step
            
  def read_data(self,paths):
    poems_sequence = []
    poems = pd.read_csv(paths[0])
    i = 1
    while True:
            poem_i = poems[poems['poem_id'] == i]
            # print(poem_i)
            index_i = poems.index[poems['poem_id'] == i]
            current_poem = ""
            for p in index_i:
                v_position = poem_i.loc[p,"v_position"]
                verse =  poem_i.loc[p,"poem_text"]
                # print(verse)
                current_poem += verse 
                if v_position == 0:
                    current_poem += " \t"
                if v_position == 1:
                    current_poem += " \n"
            if len(current_poem)>0:
              poems_sequence.append(current_poem)
            i += 1
            if i>599 :
              break
    for path in paths[1:]:
        
        poems = pd.read_csv(path)
        # print(poems['poem_id'])
        i = 1
        while True:
            poem_i = poems[poems['poem_id'] == i]
          
            index_i = poems.index[poems['poem_id'] == i]
            current_poem = ""
            for p in index_i:
                v_position = poem_i.loc[p,"v_position"]
                verse =  poem_i.loc[p,"poem_text"]
                current_poem += verse 
                if v_position == 0:
                    current_poem += " \t"
                if v_position == 1:
                    current_poem += " \n"
            poems_sequence.append(current_poem)
            i += 1
            if len(poem_i)<1 :
              break
    return poems_sequence
import os
paths = os.listdir("/content/drive/MyDrive/Data/poems")
# print(paths)
paths_arr = ["/content/drive/MyDrive/Data/poems/" + f for f in paths]
d_load = Data_load(60,2)
# poems= d_load.read_data(["/content/drive/MyDrive/Data/train.csv"]+paths_arr)
poems= d_load.read_data( ["/content/drive/MyDrive/Data/train.csv"
                        ])


X , Y  = [], []
# persian_poems = open("/content/drive/MyDrive/persian_poems_test.txt",'w')
# for poem in poems:
#   for vers in poem.split("\n"):
#     try:
#       x,y = vers.split("\t")
#       X.append(x.strip())
#       Y.append(y.strip())
#     except ValueError:
#       continue
# beyt
for poem in poems:
  verses =poem.split("\n") 
  for i in range(len(verses)-1):
    try:
      X.append(verses[i])
      Y.append(verses[i+1])
    except ValueError:
      continue
# for poem in poems:
#   index =poem.index("\t") 
#   X.append(poem[:index])
#   Y.append(poem[index:])
   

In [ ]:
df = pd.DataFrame({"poet":X[:60000],"poem":Y[:60000]})
df["text"] =df["poet"] + "<sep>" + df["poem"]
df = df.reset_index(drop=True)
df.head()

In [ ]:
poets = list(df["poet"].unique())

In [ ]:
idx = np.random.randint(0, len(df))
print(idx)
print(df.iloc[idx]["poet"])
print(df.iloc[idx]["poem"])
print(df.iloc[idx]["text"])

In [ ]:
df.info()

In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path

import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config

from IPython import display

In [ ]:
model_name_or_path = "gpt2-medium"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    bos_token='<s>', 
    eos_token='</s>', 
    pad_token='<pad>',
    unk_token='<unk>'
)
tokenizer.add_special_tokens({
    "bos_token": '<s>',
    "eos_token": '</s>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

config = AutoConfig.from_pretrained(
    model_name_or_path,
    bos_token_id=tokenizer("<s>")["input_ids"][0], 
    eos_token_id=tokenizer("</s>")["input_ids"][0], 
    pad_token_id=tokenizer("<pad>")["input_ids"][0],
    unk_token_id=tokenizer("<unk>")["input_ids"][0],
)


In [ ]:
texts = df["text"].values.tolist()

print(len(texts))

In [ ]:
max_seq = 256

In [ ]:
from torch.utils.data import Dataset  # this is the pytorch class import
import torch
torch.manual_seed(42)
class MTGDataset(Dataset):

    def __init__(self, txt_list, tokenizer, max_length=1024):

        self.tokenizer = tokenizer  # the gpt2 tokenizer we instantiated
        self.input_ids = []
        self.attn_masks = []

        for txt in txt_list:
            """
            This loop will iterate through each entry in the flavour text corpus.
            For each bit of text it will prepend it with the start of text token,
            then append the end of text token and pad to the maximum length with the 
            pad token. 
            """

            encodings_dict = tokenizer('<s>' + txt + '</s>',
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")

            """
            Each iteration then appends either the encoded tensor to a list,
            or the attention mask for that encoding to a list. The attention mask is
            a binary list of 1's or 0's which determine whether the langauge model
            should take that token into consideration or not. 
            """
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
from torch.utils.data import random_split

dataset = MTGDataset(texts, tokenizer, max_length=max_seq)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {len(train_dataset)} samples for training, and {len(val_dataset)} samples for validation testing'

In [ ]:
print(train_dataset[0][0][:100])
print(tokenizer.decode(train_dataset[5][0]))

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=8
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=8
)

In [ ]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('gpt2-medium', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("gpt2-medium", config=configuration)
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# This step is optional but will enable reproducible runs.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
epochs = 3
warmup_steps = 1e2
sample_every = 300

In [ ]:
from transformers import AdamW

# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters.
optimizer = AdamW(
    model.parameters(),
    lr=5e-4,
    eps=1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

"""
Total training steps is the number of data points, times the number of epochs. 
Essentially, epochs are training cycles, how many times each point will be seen by the model. 
"""

total_steps = len(train_dataloader) * epochs

"""
We can set a variable learning rate which will help scan larger areas of the 
problem space at higher LR earlier, then fine tune to find the exact model minima 
at lower LR later in training.
"""
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps)


In [ ]:
len(X)

In [ ]:
import random
import time
import datetime
from tqdm import tqdm


def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))


total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in tqdm(range(2), position=0):

    print(f'Beginning epoch {epoch_i + 1} of {epochs}')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every 100 batches.
        # if step % sample_every == 0 and not step == 0:

        #     elapsed = format_time(time.time() - t0)
        #     print()
        #     print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')

        #     model.eval()

        #     sample_poet = poets[np.random.randint(0, len(poets))]
        #     sample_input = f"<s>{sample_poet}<|startoftext|>"
        #     sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
        #     sample_input_ids = sample_input_ids.to(device)

        #     sample_outputs = model.generate(
        #         input_ids=sample_input_ids,
        #         # bos_token_id=random.randint(1, len(tokenizer.get_vocab())),
        #         do_sample=True,
        #         top_k=50,
        #         max_length=50,
        #         top_p=0.95,
        #         num_return_sequences=1
        #     )
        #     for i, sample_output in enumerate(sample_outputs):
        #         gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
        #         gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
        #         gen_sample_output = gen_sample_output.replace("<s>", "")
        #         gen_sample_output = gen_sample_output.replace("</s>", "")
        #         gen_sample_output = gen_sample_output.replace("<sep>", "\n")

        #         print(f'Example output: {gen_sample_output}')

            # model.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print()
    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')
    print()

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in tqdm(validation_dataloader, total=len(validation_dataloader), position=0):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs = model(b_input_ids, attention_mask=b_masks, labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print()
    print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')
    print()

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print(f'Total training took {format_time(time.time()-total_t0)}')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('precision', 2)
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

In [ ]:
import os

output_dir = '/content/drive/MyDrive/gpt2-poetry_small'
os.makedirs(output_dir, exist_ok=True)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
configuration.save_pretrained(output_dir)
!cp /content/drive/MyDrive/gpt2/tokenizer.json /content/drive/MyDrive/gpt2-fa-poetry/

In [ ]:
# from transformers import TFAutoModelForCausalLM


# tf_model = TFAutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/gpt2-fa-poetry/", from_pt=True)
# print(tf_model.summary())
# tf_model.save_pretrained("/content/drive/MyDrive/gpt2-fa-poetry-tf")
# !cp /content/drive/MyDrive/gpt2-fa-poetry-tf/tf_model.h5 /content/gpt2-fa-poetry 
# !rm -rf /content/drive/MyDrive/gpt2-fa-poetry-tf

In [ ]:
!zip -r gpt2-fa-poetry.zip gpt2-fa-poetry

In [ ]:
model_dir = '/content/drive/MyDrive/gpt2-poetry_small'
configuration = GPT2Config.from_pretrained(model_dir, output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained(model_dir, config=configuration)
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/gpt2-fa-poetry')

tokenizer = AutoTokenizer.from_pretrained(
    model_dir,
    bos_token='<s>', 
    eos_token='</s>', 
    pad_token='<pad>',
    unk_token='<unk>'
)
tokenizer.add_special_tokens({
    "bos_token": '<s>',
    "eos_token": '</s>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

config = AutoConfig.from_pretrained(
  model_dir,
    bos_token_id=tokenizer("<s>")["input_ids"][0], 
    eos_token_id=tokenizer("</s>")["input_ids"][0], 
    pad_token_id=tokenizer("<pad>")["input_ids"][0],
    unk_token_id=tokenizer("<unk>")["input_ids"][0],
)

model.resize_token_embeddings(len(tokenizer))


In [ ]:
import re
def generator(poet, max_length=20, num_return_sequences=3):
    model.eval()
    prompt = f"<s>{poet}<sep>"
    # print(prompt)

    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    # generated = generated.to(device)

    decoded_outputs = model.generate(
        generated,
        do_sample=True,
        top_k=10,
        max_length=100, 
        top_p=0.8,
        num_return_sequences=num_return_sequences,
        bos_token_id=tokenizer("<s>")["input_ids"][0], 
        eos_token_id=tokenizer("</s>")["input_ids"][0], 
        pad_token_id=tokenizer("<pad>")["input_ids"][0],
        unk_token_id=tokenizer("<unk>")["input_ids"][0], 
    )
    outputs = []
    for i, output in enumerate(decoded_outputs):
        o = tokenizer.decode(output, skip_special_tokens=False)
        # print(o)
        o = o.replace("<s>", "").replace("</s>", "").replace("<pad>","")
        outputs.append(o)
    return outputs

In [ ]:
poems = pd.read_csv("/content/drive/MyDrive/Data/test.csv")
# print(poems['poem_id'])
i = 3
poems_sequence =[]
while True:
            poem_i = poems[poems['poem_id'] == i]
            # print(poem_i)
            index_i = poems.index[poems['poem_id'] == i]
            current_poem = ""
            for p in index_i:
                v_position = poem_i.loc[p,"v_position"]
                verse =  poem_i.loc[p,"poem_text"]
                # print(verse)
                current_poem += verse 
                if v_position == 0:
                    current_poem += " \t"
                if v_position == 1:
                    current_poem += " \n"
            if len(current_poem)>0:
              poems_sequence.append(current_poem)
            i += 1
            if i>596 :
              break
# # print(poems)
# X , Y  = [], []
# for poem in poems_sequence:
#   verses =poem.split("\n") 
#   for i in range(len(verses)-1):
#     try:
#       X.append(verses[i])
#       Y.append(verses[i+1])
#     except ValueError:
#       continue
X,Y = [],[]
print(len(poems_sequence))
for poem in poems_sequence:
  # print(len(poem.split("\n")))
  for vers in poem.split("\n"):
    try:
      x,y = vers.split("\t")
      X.append(x.strip())
      Y.append(y.strip())
    except ValueError:
      continue

BLEU

In [ ]:

from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
score1 = 0
score2 = 0
score3 = 0
for x, y in zip(X[:500],Y[:500]):
  outputs = generator(x, num_return_sequences=1)
  print(x)
  print("************")
  print(y)
  print("******************")
  print(outputs[0][outputs[0].index("<sep>"):])
  print("#####################################")
  # print(outputs[0][outputs[0].index("<sep>")+15:])
  # bleu = BLEU(test_text=outputs[0][outputs[0].index("<|startoftext|>")+15:],real_text=y,gram = 1)
  # print(bleu.get_bleu())
  s1 = sentence_bleu([f.split(" ")for f in Y[:500]],outputs[0][outputs[0].index("<sep>")+15:].split(" "),weights= (1,0,0,0)) 
  s2 = sentence_bleu([f.split(" ")for f in Y[:500]],outputs[0][outputs[0].index("<sep>")+15:].split(" "),weights= (0,1,0,0)) 
  s3 = sentence_bleu([f.split(" ")for f in Y[:500]],outputs[0][outputs[0].index("<sep>")+15:].split(" "),weights= (0,0,1,0)) 
  score1 += s1
  score2 += s2
  score3 += s3
  print(s1,s2,s3)
  print("****************************************")

In [ ]:
score1/500,score2/500, score3/500

